In [1]:
# Dependencies
import csv
import json
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Create file path for josn file
jsonFilePath = 'data/data.json'

In [3]:
#Create nested dictionaries from csv file
filePath = open('data/judge_data.csv', 'r')

reader = csv.reader(filePath)

court_dict = {}

for row in reader:
    court_dict['name'] = row[0]
    if not 'children' in court_dict:
        court_dict['children'] = []
    court_index = [ind for ind, x in enumerate(court_dict['children']) if x['name'] == row[1]]
    if len(court_index) == 0:
        court_dict['children'].append({'name' : row[1], 'children': []})
        court_index = len(court_dict['children']) - 1
    else:
        court_index = court_index[0]       
    gender_index = [ind for ind, x in enumerate(court_dict['children'][court_index]['children']) if x['name'] == row[2]]
    if len(gender_index) == 0:
        court_dict['children'][court_index]['children'].append({'name' : row[2], 'children': []})
        gender_index = len(court_dict['children'][court_index]['children']) - 1
    else:
        gender_index = gender_index[0]
    gender_value_index = [ind for ind, x in enumerate(court_dict['children'][court_index]['children'][gender_index]['children']) if x['name'] == row[3]]
    if len(gender_value_index) == 0:
        court_dict['children'][court_index]['children'][gender_index]['children'].append({'name' : row[3], 'children': []})
        gender_value_index = len(court_dict['children'][court_index]['children'][gender_index]['children']) - 1
    else:
        gender_value_index = gender_value_index[0]
    race_index = [ind for ind, x in enumerate(court_dict['children'][court_index]['children'][gender_index]['children'][gender_value_index]['children']) if x['name'] == row[4]]
    if len(race_index) == 0:
        court_dict['children'][court_index]['children'][gender_index]['children'][gender_value_index]['children'].append({'name' : row[4], 'children': []})
        race_index = len(court_dict['children'][court_index]['children'][gender_index]['children'][gender_value_index]['children']) - 1
    else:
        race_index = race_index[0]
    race_value_index = [ind for ind, x in enumerate(court_dict['children'][court_index]['children'][gender_index]['children'][gender_value_index]['children'][race_index]['children']) if x['name'] == row[5]]
    if len(race_value_index) == 0:
        court_dict['children'][court_index]['children'][gender_index]['children'][gender_value_index]['children'][race_index]['children'].append({'name' : row[5], 'children': []})
        race_value_index = len(court_dict['children'][court_index]['children'][gender_index]['children'][gender_value_index]['children'][race_index]['children']) - 1
    else:
        race_value_index = race_value_index[0]
    judge_index = [ind for ind, x in enumerate(court_dict['children'][court_index]['children'][gender_index]['children'][gender_value_index]['children'][race_index]['children'][race_value_index]['children']) if x['name'] == row[6]]
    if len(judge_index) == 0:
        court_dict['children'][court_index]['children'][gender_index]['children'][gender_value_index]['children'][race_index]['children'][race_value_index]['children'].append({'name' : row[6], 'size': row[7]})
        judge_index = len(court_dict['children'][court_index]['children'][gender_index]['children'][gender_value_index]['children'][race_index]['children'][race_value_index]['children']) - 1
    else:
        judge_index = judge_index[0]

In [4]:
# Create new json file and write to it
with open(jsonFilePath, 'w') as jsonFile:
    jsonFile.write(json.dumps(court_dict, indent=4))

In [5]:
# Extract CSV into DataFrames
courts_file = 'data/court_demographics.csv'
courts_df = pd.read_csv(courts_file)
courts_df.head()

,jid,nid,last_name,first_name,middle_name,suffix,birth_month,birth_day,birth_year,birth_city,birth_state,gender,race_or_ethnicity,court_type
0,3563,1394646,Abrams,Leslie,Joyce,,NaN,NaN,1974,Madison,WI,Female,African American,U.S. District Court
1,3419,1393931,Abrams,Ronnie,,,NaN,NaN,1968,New York,NY,Female,White,U.S. District Court
2,1,1376976,Abruzzo,Matthew,T.,,4.0,30.0,1889,Brooklyn,NY,Male,White,U.S. District Court
3,2,1376981,Acheson,Marcus,Wilson,,6.0,7.0,1828,Washington,PA,Male,White,U.S. District Court
4,3,1376986,Acker,William,Marsh,Jr.,10.0,25.0,1927,Birmingham,AL,Male,White,U.S. District Court


In [6]:
# Connection to the database
connection_string = "postgres:Hercules19!@localhost:5432/courts_db"
engine = create_engine(f'postgresql://{connection_string}')

In [7]:
# Confirm tables
engine.table_names()

['courts']

In [8]:
# Write dataframe into table in database
courts_df.to_sql(name='courts', con=engine, if_exists='append', index=False)